In [1]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [2]:
import mlflow
mlflow.set_tracking_uri("http://ec2-3-82-21-172.compute-1.amazonaws.com:5000/")

In [3]:
mlflow.set_experiment("Exp 4 - Handling Imbalanced Data")

<Experiment: artifact_location='s3://mlflowexperiment-26/4', creation_time=1767609571786, experiment_id='4', last_update_time=1767609571786, lifecycle_stage='active', name='Exp 4 - Handling Imbalanced Data', tags={}>

In [4]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [5]:
def run_imbalanced_experiment(imbalance_method):
    ngram_range = (1, 3) 
    max_features = 10000 

    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
    X_train_vec = vectorizer.fit_transform(X_train)  
    X_test_vec = vectorizer.transform(X_test) 

    if imbalance_method == 'class_weights':
        class_weight = 'balanced'
    else:
        class_weight = None 
        
        if imbalance_method == 'oversampling':
            smote = SMOTE(random_state=42)
            X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'adasyn':
            adasyn = ADASYN(random_state=42)
            X_train_vec, y_train = adasyn.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'undersampling':
            rus = RandomUnderSampler(random_state=42)
            X_train_vec, y_train = rus.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'smote_enn':
            smote_enn = SMOTEENN(random_state=42)
            X_train_vec, y_train = smote_enn.fit_resample(X_train_vec, y_train)

    with mlflow.start_run() as run:
        mlflow.set_tag("mlflow.runName", f"Imbalance_{imbalance_method}_RandomForest_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "imbalance_handling")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        mlflow.set_tag("description", f"RandomForest with TF-IDF Trigrams, imbalance handling method={imbalance_method}")

        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("imbalance_method", imbalance_method)

        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42, class_weight=class_weight)
        model.fit(X_train_vec, y_train)

        y_pred = model.predict(X_test_vec)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, Imbalance={imbalance_method}")
        confusion_matrix_filename = f"confusion_matrix_{imbalance_method}.png"
        plt.savefig(confusion_matrix_filename)
        mlflow.log_artifact(confusion_matrix_filename)
        plt.close()

        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_imbalance_{imbalance_method}")

imbalance_methods = ['class_weights', 'oversampling', 'adasyn', 'undersampling', 'smote_enn']

for method in imbalance_methods:
    run_imbalanced_experiment(method)

2026/01/05 18:56:00 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Imbalance_class_weights_RandomForest_TFIDF_Trigrams at: http://ec2-3-82-21-172.compute-1.amazonaws.com:5000/#/experiments/4/runs/944bc7b56f574945abc2ea4b014d8c63
🧪 View experiment at: http://ec2-3-82-21-172.compute-1.amazonaws.com:5000/#/experiments/4


2026/01/05 18:57:29 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Imbalance_oversampling_RandomForest_TFIDF_Trigrams at: http://ec2-3-82-21-172.compute-1.amazonaws.com:5000/#/experiments/4/runs/45456c847bb544229cff71400068113b
🧪 View experiment at: http://ec2-3-82-21-172.compute-1.amazonaws.com:5000/#/experiments/4


2026/01/05 18:58:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Imbalance_adasyn_RandomForest_TFIDF_Trigrams at: http://ec2-3-82-21-172.compute-1.amazonaws.com:5000/#/experiments/4/runs/261b9061b7a54313b28ea36cb4948bd9
🧪 View experiment at: http://ec2-3-82-21-172.compute-1.amazonaws.com:5000/#/experiments/4


2026/01/05 19:00:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Imbalance_undersampling_RandomForest_TFIDF_Trigrams at: http://ec2-3-82-21-172.compute-1.amazonaws.com:5000/#/experiments/4/runs/f5e7f9600f7d40b99400060401e316cd
🧪 View experiment at: http://ec2-3-82-21-172.compute-1.amazonaws.com:5000/#/experiments/4


2026/01/05 19:02:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Imbalance_smote_enn_RandomForest_TFIDF_Trigrams at: http://ec2-3-82-21-172.compute-1.amazonaws.com:5000/#/experiments/4/runs/c622f1177e094b53a481a0e1ee237b66
🧪 View experiment at: http://ec2-3-82-21-172.compute-1.amazonaws.com:5000/#/experiments/4
